# Miscellaneous

In [1]:
# | default_exp misc

In [2]:
# | export
import time
from torch_snippets.logger import Debug, Excep, debug_mode, Info, Trace
from torch_snippets.markup2 import AD
from functools import wraps
from fastcore.basics import ifnone
from fastcore.foundation import L

In [3]:
# | export
# | hide


class Timer:
    def __init__(self, N, smooth=True, mode=1):
        "print elapsed time every iteration and print out remaining time"
        "assumes this timer is called exactly N times or less"
        self.tok = self.start = time.time()
        self.N = N
        self.ix = 0
        self.smooth = smooth
        self.mode = mode
        # 0 = instant-speed, i.e., time remaining is a funciton of only the last iteration
        # Useful when you know that each loop takes unequal time (increasing/decreasing speeds)
        # 1 = average, i.e., time remaining is a function of average of all iterations
        # Usefule when you know on average each loop or a group of loops take around the same time

    def __call__(self, ix=None, info=None):
        ix = self.ix if ix is None else ix
        info = "" if info is None else f"{info}\t"
        tik = time.time()
        elapsed = tik - self.start

        if self.mode == 0:
            ielapsed = tik - self.tok
            ispeed = ielapsed
            iremaining = (self.N - (ix + 1)) * ispeed

            iunit = "s/iter"
            if ispeed < 1:
                ispeed = 1 / ispeed
                iunit = "iters/s"
            iestimate = iremaining + elapsed
            _info = f"{info}{ix+1}/{self.N} ({elapsed:.2f}s - {iremaining:.2f}s remaining - {ispeed:.2f} {iunit})"

        else:
            speed = elapsed / (ix + 1)
            remaining = (self.N - (ix + 1)) * speed
            unit = "s/iter"
            if speed < 1:
                speed = 1 / speed
                unit = "iters/s"
            estimate = remaining + elapsed
            # print(f'N={self.N} e={elapsed:.2f} _rem={self.N-(ix+1)} r={remaining:.2f} s={speed:.2f}\t\t')
            _info = f"{info}{ix+1}/{self.N} ({elapsed:.2f}s - {remaining:.2f}s remaining - {speed:.2f} {unit})"

        print(
            _info + " " * 10,
            end="\r",
        )
        self.ix += 1
        self.tok = tik


def track2(iterable, *, total=None):
    try:
        total = ifnone(total, len(iterable))
    except:
        ...
    timer = Timer(total)
    for item in iterable:
        info = yield item
        timer(info=info)
        if info is not None:
            yield  # Just to ensure the send operation stops

In [4]:
time.sleep(3)

## Timer
Use timer as a standalone class so you have full control on when to call a lap (most useful in while loops)...

In [5]:
N = 50
t = Timer(N)
info = None

for i in range(N):
    time.sleep(0.1)
    t(info=info)  # Lap and present the time
    if i == N//2:
        print()
        info = f"My Info: {i*3.122}"

26/50 (2.70s - 2.49s remaining - 9.62 iters/s)          


## Track2
... or use track2 to directly track a for loop

In [6]:
N = 50
info = None

for i in (tracker := track2(range(N), total=N)):
    time.sleep(0.1)
    info = f"My Info: {i*3.122:.2f}"
    if i == N // 2:
        print()
    if i >= N // 2:
        tracker.send(info)

25/50 (2.59s - 2.59s remaining - 9.66 iters/s)          


Warning! NEVER RUN `tracker.send(None)` as this will skip variables silently

In [20]:
# https://youtu.be/XcU-a-eksMA
from IPython.display import YouTubeVideo

# Replace 'video_id' with the actual YouTube video ID
YouTubeVideo('XcU-a-eksMA', width=560, height=315)

In [7]:
# | export
# | hide


def summarize_input(args, kwargs, outputs=None):
    o = AD(args, kwargs)
    if outputs is not None:
        o.outputs = outputs
    return o.summary()


def timeit(func):
    def inner(*args, **kwargs):
        s = time.time()
        o = func(*args, **kwargs)
        Info(f"{time.time() - s:.2f} seconds to execute `{func.__name__}`")
        return o

    return inner


def io(func=None, *, level="debug"):
    logfuncs = {
        "debug": lambda i: Debug(i, depth=1),
        "info": lambda i: Info(i, depth=1),
        "trace": lambda i: Trace(i, depth=1),
    }
    try:
        logfunc = logfuncs[level.lower()]
    except KeyError:
        raise ValueError(f"level should be one of {list(logfuncs.keys())}")

    def decorator(func):
        def inner(*args, **kwargs):
            s = time.time()
            o = func(*args, **kwargs)
            info = f"""
{time.time() - s:.2f} seconds to execute `{func.__name__}`
{summarize_input(args=args, kwargs=kwargs, outputs=o)}
            """
            logfunc(info)
            return o

        return inner

    if func is None:
        return decorator
    else:
        return decorator(func)

## Print execution time
`@timeit` decorates and prints time taken to execute a function as an info log

In [8]:
@timeit
def foo(a, b=None):
    if b is None:
        return a + 1
    else:
        time.sleep(2)
        return a + b

In [9]:
foo(10)
foo(10, b=20)

[08/24/24 12:46:23] INFO     0.00 seconds to execute `foo`                                                                                            ]8;id=301033;file:///var/folders/1_/71dqv9vx2750gmyz77q_f45w0000gn/T/ipykernel_45033/3503404687.py:16\3503404687.py]8;;\:]8;id=833064;file:///var/folders/1_/71dqv9vx2750gmyz77q_f45w0000gn/T/ipykernel_45033/3503404687.py:16#inner:16\inner:16]8;;\

[08/24/24 12:46:25] INFO     2.01 seconds to execute `foo`                                                                                            ]8;id=736278;file:///var/folders/1_/71dqv9vx2750gmyz77q_f45w0000gn/T/ipykernel_45033/3503404687.py:16\3503404687.py]8;;\:]8;id=952859;file:///var/folders/1_/71dqv9vx2750gmyz77q_f45w0000gn/T/ipykernel_45033/3503404687.py:16#inner:16\inner:16]8;;\

30

## Print IO
`@io` will decorate to show inputs and outputs (along with time taken to execute) as a debug log

In [10]:
@io
def foo(a, b=None):
    if b is None:
        return a + 1
    else:
        time.sleep(2)
        return a + b


with debug_mode():
    foo(10)
    foo(10, b=20)

[08/24/24 12:46:26] DEBUG                                                                                                                             ]8;id=778421;file:///var/folders/1_/71dqv9vx2750gmyz77q_f45w0000gn/T/ipykernel_45033/3503404687.py:41\3503404687.py]8;;\:]8;id=712280;file:///var/folders/1_/71dqv9vx2750gmyz77q_f45w0000gn/T/ipykernel_45033/3503404687.py:41#inner:41\inner:41]8;;\
                             0.00 seconds to execute `foo`                                                                                                                  
                             args()                                                                                                                                         
                               0 - 10 (🏷️ int)                                                                                                                               
                             kwargs                                                                                                                                         
                             outputs - 11 (🏷️ int)                                                                                                                           
                                                                                                                                                                            
                                                                                                                                                                            

[08/24/24 12:46:28] DEBUG                                                                                                                             ]8;id=34123;file:///var/folders/1_/71dqv9vx2750gmyz77q_f45w0000gn/T/ipykernel_45033/3503404687.py:41\3503404687.py]8;;\:]8;id=838060;file:///var/folders/1_/71dqv9vx2750gmyz77q_f45w0000gn/T/ipykernel_45033/3503404687.py:41#inner:41\inner:41]8;;\
                             2.01 seconds to execute `foo`                                                                                                                  
                             args()                                                                                                                                         
                               0 - 10 (🏷️ int)                                                                                                                               
                             kwargs                                                                                                                                         
                               b - 20 (🏷️ int)                                                                                                                               
                             outputs - 30 (🏷️ int)                                                                                                                           
                                                                                                                                                                            
                                                                                                                                                                            

`@io` can be forced to print as log/trace if needed

In [11]:
# | export
# | hide


def tryy(
    func=None,
    *,
    output_to_return_on_fail=None,
    silence_errors=False,
    print_traceback=False,
    store_errors: bool = True,
):
    def decorator(f):
        if isinstance(store_errors, bool) and store_errors:
            error_store = []
        elif isinstance(store_errors, (list, L)):
            error_store = store_errors

        def inner(*args, **kwargs):
            try:
                return f(*args, **kwargs)
            except Exception as e:
                if not print_traceback:
                    tb = f"{type(e).__name__}: {str(e)}"
                else:
                    import traceback

                    tb = traceback.format_exc()
                if not silence_errors:
                    Excep(
                        f"Error for `{f.__name__}` with \n{summarize_input(args, kwargs)}\n{tb}"
                    )
                if store_errors is not None:
                    error_store.append(
                        AD(
                            func=f.__name__,
                            args=args,
                            kwargs=kwargs,
                            tb=tb,
                            err_type=type(e).__name__,
                        )
                    )
                return output_to_return_on_fail

        inner.F = f  # Store reference to the original function
        inner.error_store = error_store

        def error_summary():
            import pandas as pd

            return pd.DataFrame([e.dict() for e in error_store])

        inner.error_summary = error_summary
        return inner

    if callable(func):
        return decorator(func)
    return decorator

## Try Catch with a single line

In [12]:
@tryy
def do(a, b, c):
    return 1 / 0


x = do(1, 2, c=10)
assert x is None  # tryy returns None by default

[08/24/24 12:46:28] ERROR    Error for `do` with                                                                                                      ]8;id=587310;file:///var/folders/1_/71dqv9vx2750gmyz77q_f45w0000gn/T/ipykernel_45033/1057038067.py:30\1057038067.py]8;;\:]8;id=712505;file:///var/folders/1_/71dqv9vx2750gmyz77q_f45w0000gn/T/ipykernel_45033/1057038067.py:30#inner:30\inner:30]8;;\
                             args()                                                                                                                                         
                               0 - 1 (🏷️ int)                                                                                                                                
                               1 - 2 (🏷️ int)                                                                                                                                
                             kwargs                                                                                                                                         
                               c - 10 (🏷️ int)                                                                                                                               
                                                                                                                                                                            
                             ZeroDivisionError: division by zero                                                                                                            
                             Traceback (most recent call last):                                                                                                             
                                                                                                                                                                            
                               File "/var/folders/1_/71dqv9vx2750gmyz77q_f45w0000gn/T/ipykernel_45033/1057038067.py", line 21, in inner                                     
                                 return f(*args, **kwargs)                                                                                                                  
                                        |  |       -> {'c': 10}                                                                                                             
                                        |  -> (1, 2)                                                                                                                        
                                        -> <function do at 0x11ecdd800>                                                                                                     
                                                                                                                                                                            
                               File "/var/folders/1_/71dqv9vx2750gmyz77q_f45w0000gn/T/ipykernel_45033/884313129.py", line 3, in do                                          
                                 return 1 / 0                                                                                                                               
                                                                                                                                                                            
                             ZeroDivisionError: division by zero                                                                                                            
                                                                                                                                                                            
                             ╭───────────────────────────────────────── Traceback (most recent call last) ───────────────────────────

Use your own default on failure

In [13]:
@tryy(output_to_return_on_fail="😔")
def do(a, b, c):
    return 1 / 0


do(1, 2, c=10)

                    ERROR    Error for `do` with                                                                                                      ]8;id=851622;file:///var/folders/1_/71dqv9vx2750gmyz77q_f45w0000gn/T/ipykernel_45033/1057038067.py:30\1057038067.py]8;;\:]8;id=487071;file:///var/folders/1_/71dqv9vx2750gmyz77q_f45w0000gn/T/ipykernel_45033/1057038067.py:30#inner:30\inner:30]8;;\
                             args()                                                                                                                                         
                               0 - 1 (🏷️ int)                                                                                                                                
                               1 - 2 (🏷️ int)                                                                                                                                
                             kwargs                                                                                                                                         
                               c - 10 (🏷️ int)                                                                                                                               
                                                                                                                                                                            
                             ZeroDivisionError: division by zero                                                                                                            
                             Traceback (most recent call last):                                                                                                             
                                                                                                                                                                            
                               File "/var/folders/1_/71dqv9vx2750gmyz77q_f45w0000gn/T/ipykernel_45033/1057038067.py", line 21, in inner                                     
                                 return f(*args, **kwargs)                                                                                                                  
                                        |  |       -> {'c': 10}                                                                                                             
                                        |  -> (1, 2)                                                                                                                        
                                        -> <function do at 0x11ecdeac0>                                                                                                     
                                                                                                                                                                            
                               File "/var/folders/1_/71dqv9vx2750gmyz77q_f45w0000gn/T/ipykernel_45033/3352784188.py", line 3, in do                                         
                                 return 1 / 0                                                                                                                               
                                                                                                                                                                            
                             ZeroDivisionError: division by zero                                                                                                            
                                                                                                                                                                            
                             ╭───────────────────────────────────────── Traceback (most recent call last) ───────────────────────────

'😔'

Optionally print the full stacktrace if needed

In [14]:
@tryy(print_traceback=True, output_to_return_on_fail="😔")
def do(a, b, c):
    return 1 / 0


do(1, 2, c=10)

                    ERROR    Error for `do` with                                                                                                      ]8;id=53919;file:///var/folders/1_/71dqv9vx2750gmyz77q_f45w0000gn/T/ipykernel_45033/1057038067.py:30\1057038067.py]8;;\:]8;id=672838;file:///var/folders/1_/71dqv9vx2750gmyz77q_f45w0000gn/T/ipykernel_45033/1057038067.py:30#inner:30\inner:30]8;;\
                             args()                                                                                                                                         
                               0 - 1 (🏷️ int)                                                                                                                                
                               1 - 2 (🏷️ int)                                                                                                                                
                             kwargs                                                                                                                                         
                               c - 10 (🏷️ int)                                                                                                                               
                                                                                                                                                                            
                             Traceback (most recent call last):                                                                                                             
                               File "/var/folders/1_/71dqv9vx2750gmyz77q_f45w0000gn/T/ipykernel_45033/1057038067.py", line 21, in inner                                     
                                 return f(*args, **kwargs)                                                                                                                  
                                        ^^^^^^^^^^^^^^^^^^                                                                                                                  
                               File "/var/folders/1_/71dqv9vx2750gmyz77q_f45w0000gn/T/ipykernel_45033/580638143.py", line 3, in do                                          
                                 return 1 / 0                                                                                                                               
                                        ~~^~~                                                                                                                               
                             ZeroDivisionError: division by zero                                                                                                            
                                                                                                                                                                            
                             Traceback (most recent call last):                                                                                                             
                                                                                                                                                                            
                               File "/var/folders/1_/71dqv9vx2750gmyz77q_f45w0000gn/T/ipykernel_45033/1057038067.py", line 21, in inner                                     
                                 return f(*args, **kwargs)                                                                                                                  
                                        |  |       -> {'c': 10}                                                                                                             
                                        |  -> (1, 2)                                                                                  

'😔'

You can also silence the errors completely

In [15]:
@tryy(silence_errors=True, output_to_return_on_fail="😔")
def do(a, b, c):
    return 1 / 0


do(1, 2, c=10)

'😔'

You can collect all your errors in a list

In [16]:
import random

errors = []


@tryy(silence_errors=True, store_errors=errors)
def do(a, b, c):
    if random.randint(0, 100) < 50:
        return 1 / 0
    else:
        raise NotImplementedError("🤔")


for _ in range(4):
    do(1, random.randint(0, 10), c=random.randint(0, 100))

print(errors)

[
```↯ AttrDict ↯
func - do (🏷️ str)
args()
  0 - 1 (🏷️ int)
  1 - 9 (🏷️ int)
kwargs
  c - 70 (🏷️ int)
tb - NotImplementedError: 🤔 (🏷️ str)
err_type - NotImplementedError (🏷️ str)

```
, 
```↯ AttrDict ↯
func - do (🏷️ str)
args()
  0 - 1 (🏷️ int)
  1 - 3 (🏷️ int)
kwargs
  c - 19 (🏷️ int)
tb - NotImplementedError: 🤔 (🏷️ str)
err_type - NotImplementedError (🏷️ str)

```
, 
```↯ AttrDict ↯
func - do (🏷️ str)
args()
  0 - 1 (🏷️ int)
  1 - 1 (🏷️ int)
kwargs
  c - 64 (🏷️ int)
tb - ZeroDivisionError: division by zero (🏷️ str)
err_type - ZeroDivisionError (🏷️ str)

```
, 
```↯ AttrDict ↯
func - do (🏷️ str)
args()
  0 - 1 (🏷️ int)
  1 - 0 (🏷️ int)
kwargs
  c - 23 (🏷️ int)
tb - NotImplementedError: 🤔 (🏷️ str)
err_type - NotImplementedError (🏷️ str)

```
]


There's onlly one usecase where you would want to send in a list by yourself - when you want to append your errors to an existing list. The sensible default is to always store the errors, especially because this is a debugging tool.

Just access all the errors in a dataframe like so

In [17]:
import random

random.seed(10)


@tryy(silence_errors=True)
def do(a, b, c):
    if c < 50:
        return 1 / 0
    else:
        raise NotImplementedError("🤔")


for _ in range(4):
    do(1, random.randint(0, 10), c=random.randint(0, 100))

do.error_summary()

,func,args,kwargs,tb,err_type
0,do,"[1, 9]",{'c': 4},ZeroDivisionError: division by zero,ZeroDivisionError
1,do,"[1, 6]",{'c': 61},NotImplementedError: 🤔,NotImplementedError
2,do,"[1, 9]",{'c': 1},ZeroDivisionError: division by zero,ZeroDivisionError
3,do,"[1, 3]",{'c': 59},NotImplementedError: 🤔,NotImplementedError


and the actual list of errors like so

In [18]:
do.error_store

[
 ```↯ AttrDict ↯
 func - do (🏷️ str)
 args()
   0 - 1 (🏷️ int)
   1 - 9 (🏷️ int)
 kwargs
   c - 4 (🏷️ int)
 tb - ZeroDivisionError: division by zero (🏷️ str)
 err_type - ZeroDivisionError (🏷️ str)
 
 ```,
 
 ```↯ AttrDict ↯
 func - do (🏷️ str)
 args()
   0 - 1 (🏷️ int)
   1 - 6 (🏷️ int)
 kwargs
   c - 61 (🏷️ int)
 tb - NotImplementedError: 🤔 (🏷️ str)
 err_type - NotImplementedError (🏷️ str)
 
 ```,
 
 ```↯ AttrDict ↯
 func - do (🏷️ str)
 args()
   0 - 1 (🏷️ int)
   1 - 9 (🏷️ int)
 kwargs
   c - 1 (🏷️ int)
 tb - ZeroDivisionError: division by zero (🏷️ str)
 err_type - ZeroDivisionError (🏷️ str)
 
 ```,
 
 ```↯ AttrDict ↯
 func - do (🏷️ str)
 args()
   0 - 1 (🏷️ int)
   1 - 3 (🏷️ int)
 kwargs
   c - 59 (🏷️ int)
 tb - NotImplementedError: 🤔 (🏷️ str)
 err_type - NotImplementedError (🏷️ str)
 
 ```]

Finally, you want to run the function (without try) to reproduce the error and actually start debugging. Just use the `.F` attribute to access the original function that you created

In [19]:
ix = 2
data = do.error_store[ix]
do.F(*data.args, **data.kwargs)

ZeroDivisionError: division by zero

In [ ]:
def deco(decorator):
    @wraps(decorator)
    def wrapper(*args, **kwargs):
        def real_decorator(func):
            @wraps(func)
            def inner_wrapper(*fargs, **fkwargs):
                return decorator(func, *fargs, **fkwargs)

            return inner_wrapper

        if len(args) == 1 and callable(args[0]) and not kwargs:
            # Case when B is used without arguments
            return real_decorator(args[0])
        else:
            # Case when B is used with arguments
            def custom_decorator(func):
                return decorator(func, **kwargs)

            return custom_decorator

    return wrapper

In [ ]:
@deco
def B(func, *args, deco_param="default", **kwargs):
    print("B", deco_param, print("args", *args, "kwargs", **kwargs))
    return func(*args, **kwargs)

In [ ]:
@B
def C(a, b, c):
    print("C")
    return a + (b * c)


C(1, 2, 3)

In [ ]:
@B(deco_param="new_param")
def C(a, b, c):
    return a + (b * c)


# Testing
print(C(1, 2, 3))  # Outputs 'new_param' then the result 7